# Gestion d'une api

## Routes

In [8]:
from elasticsearch import Elasticsearch

class ES_connector:
    def __init__(self) -> None:
        self.es_client = None
        self.connect()

    def connect(self):
        # we can put host and port in config or env
        host = "172.31.96.1"
        port = "9200"
        es = Elasticsearch([{'host': host, 'port': port}])
        self.es_client = es
        
    def index_data(self, index_name: str, data: dict) -> dict:
        try:
            # Index the document in Elasticsearch
            response = self.es_client.index(index=index_name, document=data)
            return response
        except Exception as e:
            # Handle other errors
            raise Exception(f"Failed to index data: {str(e)}")

    def insert_document(self, index_name, document_type, document_id, document, refresh=False):
        try:
            return self.es_client.index(index=index_name, doc_type=document_type, id=document_id, body=document,
                                            refresh='wait_for', request_timeout=30)
        except Exception as e:
            print(e)

    def get_data(self, index_name, search_query, size=10): #size can come from config file
        try:
            result = self.es_client.search(index=index_name, body=search_query, allow_partial_search_results=True,
                                           size=size, request_timeout=120)
            return result
        except Exception as e:
            print(e)





In [7]:
from typing import Optional
from fastapi import FastAPI
app = FastAPI()

es = ES_connector()

total_result_per_page = 10

@app.get("/")
def home():
    return "Bienvenue sur l'API de recherche de dépôts Github"

@app.get("/search")
def search(q: str, page: Optional[int] = 1):
    search_query = {
        "query": {
            "multi_match": {
                "query": q,
                "fields": ["name", "description", "readme"]
            }
        }
    }
    result = es.get_data("github", search_query, total_result_per_page)
    return result

@app.post("/store/{index_name}")
def store_data(index_name: str, data: dict) -> dict:
    required_fields = ["name", "description", "readme"]
    missing_fields = [field for field in required_fields if field not in data]
    if missing_fields:
        raise ValueError(f"Missing required fields: {', '.join(missing_fields)}")

    try:
        # Call the ES_connector to index the data
        response = es.index_data(index_name, data)
        return {
            "message": "Document indexed successfully",
            "response": response
        }
    except Exception as e:
        raise Exception(f"Failed to index data: {str(e)}")


TypeError: NodeConfig.__init__() missing 1 required positional argument: 'scheme'

## Models

In [6]:
from pydantic import BaseModel
from typing import List, Optional

class Repo(BaseModel):
    name: str
    full_name: str
    html_url: str
    description: Optional[str] = None
    readme: str

## Dependences

In [7]:
from motor.motor_asyncio import AsyncIOMotorClient

def get_db_connection():
    client = AsyncIOMotorClient('mongodb://mongo:27017')
    return client.bdProjet

def get_elasticsearch_client():
    # Logic to get an Elasticsearch client
    pass

## Routes

In [8]:
from fastapi import FastAPI, APIRouter, HTTPException, Depends
from fastapi.middleware.cors import CORSMiddleware
from typing import List

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

app.include_router(router)

@app.get("/")
def read_root():
    return {"message": "Welcome to the FastAPI application!"}



router = APIRouter()

@router.get("/repos/", response_model=List[Repo])
async def read_repos(db=Depends(get_db_connection)):
    repos = await db.repos.find().to_list(100)
    return repos

@router.get("/repos/{repo_id}", response_model=Repo)
async def read_repo(repo_id: str, db=Depends(get_db_connection)):
    repo = await db.repos.find_one({"_id": repo_id})
    if repo is None:
        raise HTTPException(status_code=404, detail="Repo not found")
    return repo